In [1]:

import pandas as pd
import numpy as np
import string
import os
import sys
import shutil

main_ds = pd.read_json('STAR_val.json')
main = main_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]

os.chdir('./datatables3')
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

dfs = []

for csv in csv_files:
    df = pd.read_csv(csv,index_col=None)
    dfs.append(df)
    
final_df = pd.concat(dfs, ignore_index=True)

#final_df = pd.read_csv('/Users/sanjivjha/Developer/IMP-OIC/STAR_eval/all_tested.csv')
final_df = final_df.drop_duplicates()
final_df.reindex()
final_df.head()

ds = final_df
ds = ds.dropna()
ds = ds.loc[:, ~ds.columns.str.contains('^Unnamed')]
ds.head()
ds.name = 'all'

/Users/sanjivjha/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
q_ids = ds['question_id']
print(len(q_ids))
for q in q_ids:
  que = ds.query("question_id == '"+q+"'")
  answer = que['answer'].values[0].lower().strip('the ').translate(str.maketrans('', '', string.punctuation))
  choices = dict()
  choice_string = ''
  options = main_ds.query("question_id == '"+q+"'")["choices"].values[0]
  for choice in options:
    choices.update({choice['choice_id']:choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)).strip()})
    choice_string += ' ('+str(choice['choice_id']+1)+')'+str(choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)))  
  OIC_answer = int(ds.query("question_id == '"+q+"'")["OIC_answer"].values[0])
  if choices[int(OIC_answer)-1] == answer:
    ds.loc[ds['question_id'] == q, 'Match'] = 'Correct'
    
  else:
   
    ds.loc[ds['question_id'] == q, 'Match'] = 'Wrong'


575


In [3]:
interaction_ds = ds.loc[ds['question_id'].str.contains('Interaction')]
interaction_ds.name='Interaction'
interaction_ds.head()


,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match
2,Interaction_T1_76,Which object was thrown by the person?,XO8NL,3.6,12.5,The pillow.,"[{'choice_id': 0, 'choice': 'The shoe.', 'choi...",The scene opens:\nFrom now on man_f89c wearing...,3.0,Which object was thrown by the person? Guess t...,Correct
3,Interaction_T1_78,Which object was opened by the person?,OY3LS,2.5,10.1,The laptop.,"[{'choice_id': 0, 'choice': 'The laptop.', 'ch...",The scene opens:\nFrom now on glove_beed on ha...,3.0,Which object was opened by the person? Guess t...,Wrong
4,Interaction_T1_101,Which object was put down by the person?,RG0KS,17.4,27.0,The box.,"[{'choice_id': 0, 'choice': 'The cup/glass/bot...",The scene opens:\nFrom now on man_3171 wearing...,4.0,Which object was put down by the person? Guess...,Wrong
5,Interaction_T1_137,Which object was closed by the person?,NUKJ0,19.4,26.6,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",The scene opens:\nFrom now on woman_24bd weari...,3.0,Which object was closed by the person? Guess t...,Wrong
6,Interaction_T1_171,Which object was put down by the person?,LBRYS,8.9,13.4,The shoe.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",The scene opens:\nFrom now on man_753c wearing...,2.0,Which object was put down by the person? Guess...,Wrong


In [4]:
sequence_ds = ds.loc[ds['question_id'].str.contains('Sequence')]
sequence_ds.name = 'Sequence'
sequence_ds.head()

,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match
977,Sequence_T1_6,Which object did the person take after they pu...,6H78U,16.6,25.6,The towel.,"[{'choice_id': 0, 'choice': 'The towel.', 'cho...",The scene opens:\nFrom now on man_638a wearing...,3.0,Which object did the person take after they pu...,Wrong
978,Sequence_T1_19,Which object did the person put down after the...,RNLTR,7.3,16.4,The shoe.,"[{'choice_id': 0, 'choice': 'The shoe.', 'choi...",The scene opens:\nFrom now on man_05ec wearing...,1.0,Which object did the person put down after the...,Correct
979,Sequence_T1_53,Which object did the person throw after they p...,4GLAP,15.7,20.7,The shoe.,"[{'choice_id': 0, 'choice': 'The shoe.', 'choi...",The scene opens:\nFrom now on handle_096a on d...,2.0,Which object did the person throw after they p...,Wrong
980,Sequence_T1_56,Which object did the person put down after the...,XO8NL,3.6,11.6,The towel.,"[{'choice_id': 0, 'choice': 'The clothes.', 'c...",The scene opens:\nFrom now on man_a4aa wearing...,1.0,Which object did the person put down after the...,Wrong
982,Sequence_T1_171,Which object did the person put down after the...,KA09C,9.8,18.5,The shoe.,"[{'choice_id': 0, 'choice': 'The broom.', 'cho...",The scene opens:\nFrom now on woman_1e50 has h...,3.0,Which object did the person put down after the...,Correct


In [5]:
prediction_ds = ds.loc[ds['question_id'].str.contains('Prediction')]
prediction_ds.name='Prediction'
prediction_ds.head()

,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match
5263,Prediction_T1_205,What will the person do next?,YHXU9,13.6,20.725,Throw the book.,"[{'choice_id': 0, 'choice': 'Wash the window.'...",The scene opens:\nFrom now on curtain_8320 cov...,2.0,What will the person do next? Guess the most l...,Wrong
5270,Prediction_T1_469,What will the person do next?,65UVU,0.0,6.150,Put down the box.,"[{'choice_id': 0, 'choice': 'Sit on the sofa/c...",The scene opens:\nFrom now on handle_1a8e on d...,3.0,What will the person do next? Guess the most l...,Correct
5274,Prediction_T1_499,What will the person do next?,7TA23,6.6,16.300,Put down the shoe.,"[{'choice_id': 0, 'choice': 'Put down the bag....",The scene opens:\nFrom now on man_49c5 wearing...,1.0,What will the person do next? Guess the most l...,Wrong
5276,Prediction_T1_556,What will the person do next?,X8JVY,0.0,5.825,Take the book.,"[{'choice_id': 0, 'choice': 'Put down the cup/...",The scene opens:\nFrom now on window_481a in r...,2.0,What will the person do next? Guess the most l...,Correct
5279,Prediction_T1_622,What will the person do next?,BFH78,7.7,15.775,Open the book.,"[{'choice_id': 0, 'choice': 'Open the book.', ...",The scene opens:\nFrom now on book_7962 on tab...,3.0,What will the person do next? Guess the most l...,Wrong


In [6]:
feasibility_ds = ds.loc[ds['question_id'].str.contains('Feasibility')]
feasibility_ds.name= 'Feasibility'
feasibility_ds.head()

,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match
5632,Feasibility_T5_972,Which object is the person able to throw after...,E6PSM,5.8,12.5,The box.,"[{'choice_id': 0, 'choice': 'The box.', 'choic...",The scene opens:\nFrom now on hair_9562 on hea...,1.0,Which object is the person able to throw after...,Correct
5633,Feasibility_T5_973,Which object is the person able to throw after...,E6PSM,0.9,9.6,The box.,"[{'choice_id': 0, 'choice': 'The towel.', 'cho...",The scene opens:\nFrom now on handle_193c on d...,4.0,Which object is the person able to throw after...,Correct
5634,Feasibility_T5_977,Which object is the person able to put down af...,AZMVM,13.9,18.2,The pillow.,"[{'choice_id': 0, 'choice': 'The pillow.', 'ch...",The scene opens:\nFrom now on person_9433 has ...,1.0,Which object is the person able to put down af...,Correct
5635,Feasibility_T5_981,Which object is the person able to hold after ...,IO712,0.0,5.9,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",The scene opens:\nFrom now on man_c13b has hai...,2.0,Which object is the person able to hold after ...,Wrong
5636,Feasibility_T5_982,Which object is the person able to open after ...,IO712,0.0,5.9,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",The scene opens:\nFrom now on man_a312 has hai...,2.0,Which object is the person able to open after ...,Wrong


In [7]:
for i in [ds, interaction_ds,sequence_ds,prediction_ds, feasibility_ds]:
  count = (i['Match'].value_counts()['Correct'])
  length = len(i)
  print('length of {} is: {}'.format(i.name, length))
  print('Correct: {}'.format(count))
  print('Correct match percentage for {} is: {}'.format(i.name, (count/length)*100))
  
  #ds.to_csv('all_cured.csv')

length of all is: 575
Correct: 179
Correct match percentage for all is: 31.130434782608695
length of Interaction is: 239
Correct: 76
Correct match percentage for Interaction is: 31.799163179916317
length of Sequence is: 91
Correct: 33
Correct match percentage for Sequence is: 36.26373626373626
length of Prediction is: 90
Correct: 26
Correct match percentage for Prediction is: 28.888888888888886
length of Feasibility is: 155
Correct: 44
Correct match percentage for Feasibility is: 28.387096774193548
